# Exploring the Hotel-Level Data

Here, we convert all reservations to hotel nightly sales statistics using the `parse_dates` and `res_to_stats` functions in `utils.py`.

We'll start by deriving some basic information about each hotel, including:
* **Capacity** (total number of rooms)
* **Occupancy** (rooms sold / capacity)

Then, we'll pull more statistics into the `stats` DataFrames:
* **Revenue and Rooms Sold by Customer Segment**
* **ADR by Customer Segment**

These stats will help us understand what kind of hotels we're working with.

In [1]:
import pandas as pd
import numpy as np

from utils import generate_hotel_dfs

pd.options.display.max_rows = 30
pd.options.display.max_columns = 60

In [2]:
h1_res, h1_dbd = generate_hotel_dfs('../data/H1.csv')

In [3]:
h2_res, h2_dbd = generate_hotel_dfs('../data/H2.csv')

In [4]:
# Uncomment these four lines to save results from generate_hotel_dfs to save time later
h1_dbd.to_pickle("pickle/h1_dbd.pick")
h2_dbd.to_pickle("pickle/h2_dbd.pick")
h1_res.to_pickle("pickle/h1_res.pick")
h2_res.to_pickle("pickle/h2_res.pick")

# if you have already run generate_hotel_dfs and pickled the result, use this to save time.
# h1_dbd = pd.read_pickle("pickle/h1_dbd.pick")
# h2_dbd = pd.read_pickle("pickle/h2_dbd.pick")
# h1_res = pd.read_pickle("pickle/h1_res.pick")
# h2_res = pd.read_pickle("pickle/h2_res.pick")

In [ ]:
h1_res

In [ ]:
h2_res

In [ ]:
h1_dbd.head()

In [ ]:
[col for col in h1_dbd.columns]

In [ ]:

h1_dbd

In [ ]:
h1_dbd.describe()

In [ ]:
h1_dbd.fillna(0, inplace=True)

## Capacity

EW- ADD TO UTILS.PY

Based on the above tables, we can see the maximum capacity of each hotel.

**H1 (Resort Hotel)'s capacity is 187 rooms.**

**H2 (City Hotel)'s capacity is 226 rooms.**

## Get hotel statistics
* Occ, room rate, RevPAR, by month, by WD/WE
    * By segment?